In [1]:
!pip install pyspark


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.sql.streaming.schemaInference", True).getOrCreate()

stream = spark.\
    readStream.\
    format("ws").\
    option("schema", "ticker").\
    load() # we need to pass `option("schema", "ticker")` to get correct channel subscribed

query = stream.select("side", "product_id", "last_size", "best_bid", "best_ask", "time").\
    writeStream.\
    format("console").\
    outputMode("append").\
    option("truncate", "false").\
    start()

query.awaitTermination(10) # Let's wait for 10 seconds.
query.stop() # Let's stop the query
stream.printSchema()
#spark.stop() # And stop the whole session

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/07 14:01:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/07 14:01:18 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-c0413ed9-07ae-4c87-ba91-526ec10ec20d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/12/07 14:01:18 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+----------+---------+--------+--------+----+
|side|product_id|last_size|best_bid|best_ask|time|
+----+----------+---------+--------+--------+----+
+----+----------+---------+--------+--------+----+

-------------------------------------------
Batch: 1
-------------------------------------------
+----+----------+---------+--------+--------+-----------------------+
|side|product_id|last_size|best_bid|best_ask|time                   |
+----+----------+---------+--------+--------+-----------------------+
|buy |ETH-BTC   |1.0      |0.08525 |0.08528 |2021-12-07 14:01:14.136|
|sell|ETH-USD   |1.048012 |4342.84 |4343.39 |2021-12-07 14:01:21.408|
+----+----------+---------+--------+--------+-----------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+----+----------+----------+--------+--------+-----------------------+
|side|produc

-------------------------------------------
Batch: 14
-------------------------------------------
+----+----------+----------+--------+--------+-----------------------+
|side|product_id|last_size |best_bid|best_ask|time                   |
+----+----------+----------+--------+--------+-----------------------+
|buy |ETH-USD   |0.02472812|4342.78 |4342.79 |2021-12-07 14:01:26.562|
+----+----------+----------+--------+--------+-----------------------+

-------------------------------------------
Batch: 15
-------------------------------------------
+----+----------+----------+--------+--------+-----------------------+
|side|product_id|last_size |best_bid|best_ask|time                   |
+----+----------+----------+--------+--------+-----------------------+
|buy |ETH-USD   |0.01353023|4342.78 |4342.79 |2021-12-07 14:01:27.108|
+----+----------+----------+--------+--------+-----------------------+

-------------------------------------------
Batch: 16
--------------------------------------

Uruchamiająć `stream.start()` uruchamiamy w osobnym demonie websocket który streamuje wyniki. Jeżeli wystąpi jakiś błąd po stronie front-endu (np. błąd parsowania kolejnej linijki Pythona) fakt ten nie zostanie zgłoszony do sparka i socket pozostanie otwarty! Należy pamiętać, by zamykać stream za każdym razem używająć metody `stop()` (w powyższym przykładzie `query.stop()`). W przypadku utracenia referencji do zapytania, należy zastopować całą sesję również metodą `stop()` (w powyższym przykładzie `spark.stop()`) 

In [21]:
# Panic button - press only if you messed up opening new websocket and lost reference to it

query.stop()
spark.stop()

-------------------------------------------
Batch: 495
-------------------------------------------
+----------+-----------------+
|product_id|       avg(price)|
+----------+-----------------+
|   ETH-USD|4350.860754873498|
+----------+-----------------+



AttributeError: 'DataFrame' object has no attribute 'stop'

-------------------------------------------
Batch: 270
-------------------------------------------
+-------+--------------------+----------+
|  price|                time|product_id|
+-------+--------------------+----------+
|4355.81|2021-12-07 13:52:...|   ETH-USD|
|4353.89|2021-12-07 13:52:...|   ETH-USD|
|4355.02|2021-12-07 13:52:...|   ETH-USD|
|4354.01|2021-12-07 13:52:...|   ETH-USD|
|4355.02|2021-12-07 13:52:...|   ETH-USD|
|4353.64|2021-12-07 13:52:...|   ETH-USD|
|0.08535|2021-12-07 13:52:...|   ETH-BTC|
| 4353.5|2021-12-07 13:52:...|   ETH-USD|
|4355.02|2021-12-07 13:52:...|   ETH-USD|
|4353.51|2021-12-07 13:52:...|   ETH-USD|
|4355.01|2021-12-07 13:52:...|   ETH-USD|
|4353.47|2021-12-07 13:52:...|   ETH-USD|
|4355.01|2021-12-07 13:52:...|   ETH-USD|
|4353.46|2021-12-07 13:52:...|   ETH-USD|
|4355.01|2021-12-07 13:52:...|   ETH-USD|
|4354.46|2021-12-07 13:52:...|   ETH-USD|
|4354.03|2021-12-07 13:52:...|   ETH-USD|
|4354.03|2021-12-07 13:52:...|   ETH-USD|
|4353.13|2021-12-07

-------------------------------------------
Batch: 908
-------------------------------------------
+----------+-----+
|product_id|count|
+----------+-----+
|   ETH-USD| 6835|
+----------+-----+



-------------------------------------------
Batch: 496
-------------------------------------------
+----------+-----------------+
|product_id|       avg(price)|
+----------+-----------------+
|   ETH-USD|4350.863996686686|
+----------+-----------------+



...

-------------------------------------------
Batch: 917
-------------------------------------------
+----------+-----+
|product_id|count|
+----------+-----+
|   ETH-BTC|  408|
|   ETH-USD| 6880|
+----------+-----+



-------------------------------------------
Batch: 505
-------------------------------------------
+----------+-----------------+
|product_id|       avg(price)|
+----------+-----------------+
|   ETH-USD|4350.882477930611|
+----------+-----------------+



-------------------------------------------
Batch: 918
-------------------------------------------
+----------+-----+
|product_id|count|
+----------+-----+
|   ETH-USD| 6883|
+----------+-----+

-------------------------------------------
Batch: 269
-------------------------------------------
+-------+--------------------+----------+
|  price|                time|product_id|
+-------+--------------------+----------+
| 4354.1|2021-12-07 13:52:...|   ETH-USD|
|4352.03|2021-12-07 13:53:...|   ETH-USD|
|4354.52|2021-12-07 13:52:...|   ETH-USD|
|4352.03|2021-12-07 13:53:...|   ETH-USD|
| 4354.1|2021-12-07 13:52:...|   ETH-USD|
|4352.01|2021-12-07 13:53:...|   ETH-USD|
|4353.24|2021-12-07 13:52:...|   ETH-USD|
|4352.01|2021-12-07 13:53:...|   ETH-USD|
|4353.23|2021-12-07 13:52:...|   ETH-USD|
| 4351.6|2021-12-07 13:53:...|   ETH-USD|
|0.08533|2021-12-07 13:52:...|   ETH-BTC|
|0.08534|2021-12-07 13:52:...|   ETH-BTC|
|4352.64|2021-12-07 13:52:...|   ETH-USD|
|4352.65|2021-12-07 13:52:...|   E

-------------------------------------------
Batch: 506
-------------------------------------------
+----------+------------------+
|product_id|        avg(price)|
+----------+------------------+
|   ETH-USD|4350.8848842450325|
+----------+------------------+



-------------------------------------------
Batch: 919
-------------------------------------------
+----------+-----+
|product_id|count|
+----------+-----+
|   ETH-USD| 6895|
+----------+-----+



# Zadanie 1

**Analiza strumienia danych CoinBase (3p)**. Napisz zapytanie, które wypisuje średnią wartość wybranego parametru (np. `price`) w przesuwnych oknach czasowych względem czasu transakcji (kolumna `time`), grupując po relacji wymiany (z jakiej waluty na jaką walutę - kolumna `product_id`). 

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import window, avg

spark = SparkSession.builder.\
    config("spark.sql.streaming.schemaInference", True).\
    config("spark.sql.streaming.forceDeleteTempCheckpointLocation", True).getOrCreate()  

stream = spark.readStream.\
    format("ws").\
    option("schema", "ticker").\
    load()

windowDuration = '60 seconds'
slideDuration = '60 seconds'

prices = stream.select("price", "time", "product_id")

query = prices.groupBy(window(prices.time, windowDuration, slideDuration), prices.product_id).agg(avg("price").alias("avg_price"))

q = query.writeStream.\
    format("console").\
    outputMode("complete").\
    option("truncate", "false").\
    option("numRows", "10000").\
    trigger(processingTime="12 seconds").\
    start()

q.awaitTermination()

21/12/07 14:41:19 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-bcb88e73-c07f-46e2-aa4a-1f4ba42f7176. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/12/07 14:41:19 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+----------+---------+
|window|product_id|avg_price|
+------+----------+---------+
+------+----------+---------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+----------+------------------+
|window                                    |product_id|avg_price         |
+------------------------------------------+----------+------------------+
|{2021-12-07 14:41:00, 2021-12-07 14:42:00}|ETH-BTC   |0.08547           |
|{2021-12-07 14:41:00, 2021-12-07 14:42:00}|ETH-USD   |4386.6272727272735|
+------------------------------------------+----------+------------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg_price          |
+------------------------------------------+----------+-------------------+
|{2021-12-07 14:41:00, 2021-12-07 14:42:00}|ETH-BTC   |0.08548333333333334|
|{2021-12-07 14:41:00, 2021-12-07 14:42:00}|ETH-USD   |4386.857968750001  |
+------------------------------------------+----------+-------------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg_price          |
+------------------------------------------+----------+-------------------+
|{2021-12-07 14:41:00, 2021-12-07 14:42:00}|ETH-BTC   |0.08548333333333334|
|{2021-12-07 14:41:00, 2021-12-07 14:42:00}|ETH-USD   |4387.403898305085  |
+------------------------------------------+----------+-------------------+



-------------------------------------------
Batch: 4
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg_price          |
+------------------------------------------+----------+-------------------+
|{2021-12-07 14:42:00, 2021-12-07 14:43:00}|ETH-USD   |4387.308571428571  |
|{2021-12-07 14:41:00, 2021-12-07 14:42:00}|ETH-BTC   |0.08548499999999999|
|{2021-12-07 14:41:00, 2021-12-07 14:42:00}|ETH-USD   |4387.222000000001  |
+------------------------------------------+----------+-------------------+



-------------------------------------------
Batch: 5
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg_price          |
+------------------------------------------+----------+-------------------+
|{2021-12-07 14:42:00, 2021-12-07 14:43:00}|ETH-USD   |4387.758764044944  |
|{2021-12-07 14:41:00, 2021-12-07 14:42:00}|ETH-BTC   |0.08548499999999999|
|{2021-12-07 14:41:00, 2021-12-07 14:42:00}|ETH-USD   |4387.222000000001  |
+------------------------------------------+----------+-------------------+



-------------------------------------------
Batch: 6
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg_price          |
+------------------------------------------+----------+-------------------+
|{2021-12-07 14:42:00, 2021-12-07 14:43:00}|ETH-USD   |4387.581843971631  |
|{2021-12-07 14:41:00, 2021-12-07 14:42:00}|ETH-BTC   |0.08548499999999999|
|{2021-12-07 14:42:00, 2021-12-07 14:43:00}|ETH-BTC   |0.08548333333333334|
|{2021-12-07 14:41:00, 2021-12-07 14:42:00}|ETH-USD   |4387.222000000001  |
+------------------------------------------+----------+-------------------+



-------------------------------------------
Batch: 7
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg_price          |
+------------------------------------------+----------+-------------------+
|{2021-12-07 14:42:00, 2021-12-07 14:43:00}|ETH-USD   |4387.61095505618   |
|{2021-12-07 14:41:00, 2021-12-07 14:42:00}|ETH-BTC   |0.08548499999999999|
|{2021-12-07 14:42:00, 2021-12-07 14:43:00}|ETH-BTC   |0.085494           |
|{2021-12-07 14:41:00, 2021-12-07 14:42:00}|ETH-USD   |4387.222000000001  |
+------------------------------------------+----------+-------------------+



KeyboardInterrupt: 

# Zadanie 2

**Watermarking i dane opóźnione (3p).** 
Zmodyfikuj zapytanie z zadania 1 tak, aby zademonstować mechanizm znaków wodnych (watermarks) i obsługi danych opóźnionych. W konsoli powinno być widać, że aktualizują się odpowiednie wiersze tabeli wynikowej (tryb update), w szczególności aktualizacja wcześniejszych okien czasowych po przybyciu danych opóźnionych. **Do rozwiązania tego zadania proszę dołączyć przykładowy output i jego opis wyjaśniający na konkretnym przykładzie działanie znaku wodnego i danych opóźnionych**. 

Do ćwiczenia można wykorzystać skrypt w katalogu `/mock` napisany w [Scala-cli](https://scala-cli.virtuslab.org), który posłuży jako kontrolowane źródło danych CoinBase przez Websocket. 

Skrypt można uruchomić wykorzystując Docker:

```
make image
make run
```

Spowoduje to utworzenie websocketowego serwera pod adresem `ws://mock:8025`

Po uruchomieniu serwera należy wykonać poniższą komórkę, w której zapytanie czyta dane z utworzonego websocketa. Skrypt wysyła przykładowe wiadomości w formacie CoinBase co 10 sekund:

- W pierwszej serii wysyłane wiadomości o znacznikach czasowych 0s, 14s, 7s  
- W drugiej serii wysyłane są wiadomości o znacznikach czasowych 15s, 8s, 21s  
- W trzeciej serii wysyłane są wiadomości o znacznikach czasowych 4s, 17s  

Dla tych danych można ustawić okno czasowe na interwał 10 sekund. Skrypt można też zmodyfikować, tak aby wysyłał inne dane. 


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import window, avg

spark = SparkSession.builder.config("spark.sql.streaming.schemaInference", True).getOrCreate()

# Note url option!
stream = spark.readStream.\
    format("ws").\
    option("schema", "ticker").\
    option("url", "ws://mock:8025").load() # we pass explicit url option to subscribe to our mock service

windowDuration = '10 seconds'
slideDuration = '10 seconds'

prices = stream.select("price", "time", "product_id")

query = prices.withWatermark('time', '10 seconds').groupBy(window(prices.time, windowDuration, slideDuration), prices.product_id).agg(avg("price").alias("avg_price"))

q = query.writeStream.\
    format("console").\
    outputMode("complete").\
    option("truncate", "false").\
    option("numRows", "10000").\
    trigger(processingTime="10 seconds").\
    start()

q.awaitTermination()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/07 19:59:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/07 19:59:19 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-6fed97ea-707b-47ee-9864-b7b4be9bd0ea. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/12/07 19:59:19 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+----------+---------+
|window|product_id|avg_price|
+------+----------+---------+
+------+----------+---------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+----------+-----------------+
|window                                    |product_id|avg_price        |
+------------------------------------------+----------+-----------------+
|{2021-11-01 00:00:00, 2021-11-01 00:00:10}|ETH-USD   |311.6473510235851|
|{2021-11-01 00:00:10, 2021-11-01 00:00:20}|ETH-USD   |69.96249592009818|
+------------------------------------------+----------+-----------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+----------+------------------+
|window                                    |product_id|avg_price         |
+------------------------------------------+----------+------------------+
|{2021-11-01 00:00:20, 2021-11-01 00:00:30}|ETH-USD   |662.3392155083254 |
|{2021-11-01 00:00:00, 2021-11-01 00:00:10}|ETH-USD   |273.6106463669501 |
|{2021-11-01 00:00:10, 2021-11-01 00:00:20}|ETH-USD   |143.16888822352175|
+------------------------------------------+----------+------------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+----------+------------------+
|window                                    |product_id|avg_price         |
+------------------------------------------+----------+------------------+
|{2021-11-01 00:00:20, 2021-11-01 00:00:30}|ETH-USD   |662.3392155083254 |
|{2021-11-01 00:00:00, 2021-11-01 00:00:10}|ETH-USD   |223.27183611172697|
|{2021-11-01 00:00:10, 2021-11-01 00:00:20}|ETH-USD   |362.70211892101764|
+------------------------------------------+----------+------------------+



Widać, że w batchu 2 dostajemy wartość z przedziału 20-30 s., która w batchu 3 nie zostaje już zaktualizowana (zgodnie z założeniami).
W batchach 1-3 następuje uaktualnienie średniej dla pozostałych dwóch przedziałów, zgodnie z założeniami.

# Zadanie 3

**Łączenie strumieni (3p)**. Rozdziel sztucznie dane CoinBase z kanału `ticker` na dwa strumienie (wykorzystując filtrowanie subskrypcji): jeden strumień dla `side="sell"`, drugi dla `side="buy"`. Następnie stwórz zapytanie, które łączy te strumienie i wypisuje transakcje dla danego `product_id`, które występowały po sobie w ciągu 1s.

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col

spark = SparkSession.\
    builder.\
    config("spark.sql.streaming.schemaInference", True).\
    config("spark.sql.streaming.forceDeleteTempCheckpointLocation", True).\
    getOrCreate()  

stream = spark.\
    readStream.\
    format("ws").\
    option("schema", "ticker").\
    load()

side_sell = stream.select("side","product_id","price", "time").filter(col("side")=="sell").withColumnRenamed("time","sell_time").withColumnRenamed("product_id", "sell_product")
side_buy = stream.select("side","product_id","price", "time").filter(col("side")=="buy").withColumnRenamed("time","buy_time").withColumnRenamed("product_id", "buy_product")

joined = side_buy.join(
    side_sell,
    expr("""
        buy_product = sell_product AND
        buy_time >= sell_time AND
        buy_time <= sell_time + interval 1 seconds
        """)
    )

q = joined.writeStream.\
    format("console").\
    outputMode("append").\
    option("truncate", "false").\
    option("numRows", "10000").\
    trigger(processingTime="12 seconds").\
    start()

q.awaitTermination()

21/12/07 14:54:31 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-fcba3c29-4cf4-41e5-b720-8f45d9fe1e87. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/12/07 14:54:31 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----------+-----+--------+----+------------+-----+---------+
|side|buy_product|price|buy_time|side|sell_product|price|sell_time|
+----+-----------+-----+--------+----+------------+-----+---------+
+----+-----------+-----+--------+----+------------+-----+---------+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+-----------+-------+-----------------------+----+------------+-------+-----------------------+
|side|buy_product|price  |buy_time               |side|sell_product|price  |sell_time              |
+----+-----------+-------+-----------------------+----+------------+-------+-----------------------+
|buy |ETH-USD    |4396.14|2021-12-07 14:54:35.267|sell|ETH-USD     |4395.59|2021-12-07 14:54:35.036|
|buy |ETH-USD    |4395.3 |2021-12-07 14:54:35.27 |sell|ETH-USD     |4395.59|2021-12-07 14:54:35.036|
|buy |ETH-USD    |4394.36|2021-12-07 14:54:35.5  |sell|ETH-USD     |4395.59|2021-12-07 14:54:35.036|
|buy |ETH-USD    |4394.36|2021-12-07 14:54:35.809|sell|ETH-USD     |4395.59|2021-12-07 14:54:35.036|
|buy |ETH-USD    |4394.36|2021-12-07 14:54:35.5  |sell|ETH-USD     |4395.29|2021-12-07 14:54:35.31 |
|buy |ETH-USD    |4394.36|2021-12-07 14:54:35.809|sell|ETH-USD     |4395.29|2021-12-07 14:54:35

-------------------------------------------
Batch: 2
-------------------------------------------
+----+-----------+-------+-----------------------+----+------------+-------+-----------------------+
|side|buy_product|price  |buy_time               |side|sell_product|price  |sell_time              |
+----+-----------+-------+-----------------------+----+------------+-------+-----------------------+
|buy |ETH-USD    |4392.01|2021-12-07 14:54:48.022|sell|ETH-USD     |4392.8 |2021-12-07 14:54:47.295|
|buy |ETH-USD    |4392.01|2021-12-07 14:54:48.022|sell|ETH-USD     |4392.8 |2021-12-07 14:54:47.295|
|buy |ETH-USD    |4392.01|2021-12-07 14:54:48.022|sell|ETH-USD     |4392.8 |2021-12-07 14:54:47.316|
|buy |ETH-USD    |4392.01|2021-12-07 14:54:48.022|sell|ETH-USD     |4392.8 |2021-12-07 14:54:47.316|
|buy |ETH-USD    |4392.01|2021-12-07 14:54:48.022|sell|ETH-USD     |4392.59|2021-12-07 14:54:47.339|
|buy |ETH-USD    |4393.75|2021-12-07 14:54:45.884|sell|ETH-USD     |4393.49|2021-12-07 14:54:45

-------------------------------------------
Batch: 3
-------------------------------------------
+----+-----------+-------+-----------------------+----+------------+-------+-----------------------+
|side|buy_product|price  |buy_time               |side|sell_product|price  |sell_time              |
+----+-----------+-------+-----------------------+----+------------+-------+-----------------------+
|buy |ETH-BTC    |0.08538|2021-12-07 14:54:52.834|sell|ETH-BTC     |0.08536|2021-12-07 14:54:52.019|
|buy |ETH-BTC    |0.08539|2021-12-07 14:54:52.834|sell|ETH-BTC     |0.08536|2021-12-07 14:54:52.019|
|buy |ETH-USD    |4393.55|2021-12-07 14:54:52.985|sell|ETH-USD     |4393.54|2021-12-07 14:54:52.712|
|buy |ETH-USD    |4394.22|2021-12-07 14:54:53.627|sell|ETH-USD     |4393.54|2021-12-07 14:54:52.712|
|buy |ETH-USD    |4394.22|2021-12-07 14:54:53.685|sell|ETH-USD     |4393.54|2021-12-07 14:54:52.712|
|buy |ETH-USD    |4394.36|2021-12-07 14:54:53.685|sell|ETH-USD     |4393.54|2021-12-07 14:54:52

KeyboardInterrupt: 